# Deep Neural Net for Sentiment Classification

This Deep Net performs sentiment analysis on the IMDB review dataset.

In [ ]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#### Set Hyperparameters

In [ ]:
output_dir = 'model_output/dense'

epochs = 4
batch_size = 128

n_dim = 64
n_unique_words = 5000
n_words_to_skip = 50
max_review_length = 100
pad_type = trunc_type = 'pre'

n_dense = 64
dropout = 0.5

#### Load Data

In [ ]:
(X_train, y_train), (X_valid, y_valid) = imdb.load_data(num_words=n_unique_words, skip_top=n_words_to_skip)

In [ ]:
X_train[0]

In [ ]:
X_train[0:6]

In [ ]:
for x in X_train[0:6]:
    print(len(x))

In [ ]:
y_train[0:6]

In [ ]:
len(X_train), len(y_train)

#### Restore Words from index

In [ ]:
word_index = keras.datasets.imdb.get_word_index()
word_index = {k : (v+3) for k,v in word_index.items()}
word_index['PAD'] = 0
word_index['START'] = 1
word_index['UNK'] = 2

In [ ]:
word_index

In [ ]:
index_word = {v:k for k,v in word_index.items()}

In [ ]:
X_train[0]

In [ ]:
' '.join(index_word[id] for id in X_train[0])

In [ ]:
(all_X_train, _), (all_X_valid, _) = imdb.load_data()

In [ ]:
' '.join(index_word[id] for id in all_X_train[0])

#### Preprocess Data

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
X_valid = pad_sequences(X_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

In [ ]:
X_train[0:6]

In [ ]:
for x in X_train[0:6]:
    print(len(x))

In [ ]:
' '.join(index_word[id] for id in X_train[0])

In [ ]:
' '.join(index_word[id] for id in X_train[5])

#### Design Deep Net Architecture

In [ ]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length))
model.add(Flatten())
model.add(Dense(n_dense, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
# Embedding 1
n_dim, n_unique_words, n_dim*n_unique_words

In [ ]:
# Flatten 1
max_review_length, n_dim, n_dim*max_review_length

In [ ]:
# Dense 1 (Fully Connected)
n_dense, n_dim*max_review_length*n_dense + n_dense

In [ ]:
# Dense 2
n_dense + 1

#### Configure the Model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+'/weights.{epoch:02d}.hdf5')

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Train the Model

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_valid, y_valid))#, callbacks=[modelcheckpoint])

#### Evaluate

In [ ]:
#model.load_weights(output_dir+'/weights.01.hdf5')

In [ ]:
y_hat = model.predict_proba(X_valid)

In [ ]:
y_hat[0]

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
pct_auc = roc_auc_score(y_valid, y_hat) * 100

In [ ]:
'{:0.2f}'.format(pct_auc)

In [ ]:
float_y_hat = []
for y in y_hat:
    float_y_hat.append(y[0])

In [ ]:
ydf = pd.DataFrame(list(zip(float_y_hat, y_valid)), columns=['y_hat', 'y'])

In [ ]:
ydf.head(10)

In [ ]:
' '.join(index_word[id] for id in all_X_valid[0])

In [ ]:
' '.join(index_word[id] for id in all_X_valid[6])

In [ ]:
ydf[(ydf.y == 0) & (ydf.y_hat > 0.9)].head(10)

In [ ]:
' '.join(index_word[id] for id in all_X_valid[25])

In [ ]:
ydf[(ydf.y == 1) & (ydf.y_hat < 0.1)].head(10)

In [ ]:
' '.join(index_word[id] for id in all_X_valid[151])